In [1]:
import re
import urllib
import pdftotext
from os import walk
import PyPDF2 
import textract
import pandas as pd 
import fitz # PyMuPDF
import io
import pytesseract
from PIL import Image

In [184]:
mypath = "diarios"
f = []
names = []
for (dirpath, dirnames, filenames) in walk(mypath):
    [f.append(f"{dirpath}/{file}") for file in filenames]
    names.append(dirnames)
#for file in f:
 #   with open(file,'rb') as f:
  #      pdfReader = PyPDF2.PdfFileReader(f)
   #     pageObj = pdfReader.getPage(0)     

In [177]:
print(type(pytesseract.image_to_string(Image.open('image3_2.png'), lang='por' )) ) # Extraindo o texto da imagem


<class 'str'>


In [187]:
names[0]

['2020-10-16',
 '2020-10-29',
 '2020-10-20',
 '2020-10-27',
 '2020-10-26',
 '2020-10-19',
 '2020-10-21',
 '2020-10-28',
 '2020-11-19',
 '2020-11-18',
 '2020-11-05',
 '2020-11-03',
 '2020-11-04',
 '2020-10-03',
 '2020-10-05',
 '2020-10-02',
 '2020-10-15',
 '2020-10-23',
 '2020-10-22',
 '2020-10-13',
 '2020-10-14',
 '2020-11-06',
 '2020-11-07',
 '2020-10-09',
 '2020-10-07',
 '2020-10-01',
 '2020-10-06',
 '2020-10-08']

In [193]:
# iterate over PDF pages
for file,name in zip(f,names[0]):
    pdf_file = fitz.open(file)
    text = ""
    for page_index in range(len(pdf_file)):
        # get the page itself
        page = pdf_file[page_index]
        image_list = page.getImageList()
        # printing number of images found in this page
        if image_list:
            print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
        else:
            print("[!] No images found on page", page_index)
        for image_index, img in enumerate(page.getImageList(), start=1):
            # get the XREF of the image
            xref = img[0]
            # extract the image bytes
            base_image = pdf_file.extractImage(xref)
            image_bytes = base_image["image"]
            # get the image extension
            image_ext = base_image["ext"]
            # load it to PIL
            image = Image.open(io.BytesIO(image_bytes))
            # save it to local disk
            text += pytesseract.image_to_string(image, lang='por')  # Extraindo o texto da imagem

            #image.safve(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb"))
        file = open(f"outputs/{name}.txt","w")
        file.write(text)
        file.close()
        #print(text)

[+] Found a total of 2 images in page 0
[+] Found a total of 3 images in page 0
[+] Found a total of 3 images in page 1
[+] Found a total of 3 images in page 2
[+] Found a total of 3 images in page 3
[+] Found a total of 1 images in page 4
[+] Found a total of 2 images in page 5
[+] Found a total of 2 images in page 0
[+] Found a total of 2 images in page 1
[+] Found a total of 2 images in page 2
[+] Found a total of 1 images in page 3
[+] Found a total of 2 images in page 0
[+] Found a total of 2 images in page 1
[+] Found a total of 2 images in page 2
[+] Found a total of 2 images in page 3
[+] Found a total of 2 images in page 4
[+] Found a total of 2 images in page 0
[+] Found a total of 2 images in page 1
[+] Found a total of 2 images in page 0
[+] Found a total of 2 images in page 1
[+] Found a total of 3 images in page 2
[+] Found a total of 3 images in page 3
[+] Found a total of 3 images in page 4
[+] Found a total of 2 images in page 5
[+] Found a total of 1 images in page 6


In [4]:
def create_dataframe(dirpath,filename):
    print(f"{dirpath}{filename}")
    gazette = dict()
    with open(filename, 'r') as file:
        gazette['date'] = filename[8:-4] #pega só a data no nome do arquivo.
        gazette['content'] = re.sub('\s+', ' ', file.read())
    #print(gazette)
    return gazette
df_gazettes = pd.DataFrame()
for (dirpath, dirnames, filenames) in walk("outputs/"):
    df_gazettes = df_gazettes.append([create_dataframe(dirpath,f"{dirpath}{file}") for file in filenames])

outputs/outputs/2020-11-07.txt
outputs/outputs/2020-11-06.txt
outputs/outputs/2020-11-04.txt
outputs/outputs/2020-11-05.txt
outputs/outputs/2020-11-03.txt
outputs/outputs/2020-10-19.txt
outputs/outputs/2020-10-27.txt
outputs/outputs/gazettes.csv
outputs/outputs/2020-10-26.txt
outputs/outputs/2020-10-22.txt
outputs/outputs/2020-10-23.txt
outputs/outputs/2020-10-09.txt
outputs/outputs/2020-10-21.txt
outputs/outputs/2020-10-20.txt
outputs/outputs/2020-10-08.txt
outputs/outputs/2020-10-05.txt
outputs/outputs/2020-10-06.txt
outputs/outputs/2020-10-07.txt
outputs/outputs/2020-10-13.txt
outputs/outputs/2020-10-03.txt
outputs/outputs/2020-10-02.txt
outputs/outputs/2020-10-16.txt
outputs/outputs/2020-10-28.txt
outputs/outputs/2020-10-14.txt
outputs/outputs/2020-10-15.txt
outputs/outputs/2020-10-01.txt
outputs/outputs/2020-10-29.txt
outputs/outputs/2020-11-19.txt
outputs/outputs/2020-11-18.txt


In [278]:
df_gazettes.to_csv('outputs/gazettes.csv',index=False)

In [277]:
df_gazettes

,date,content
0,2020-11-07,t Ã â 7 Z Prefeitura Municipal da Estância Tur...
1,2020-11-06,IMPUGNAÇÃO À PREFEITURA MUNICIPAL DA ESTÂNCIA ...
2,2020-11-04,"/""“'N_ DECRETO Nº 9.061, DE 05 DE OUTUBRO DE ..."
3,2020-11-05,"t Ã â 7 Z LEI MUNICIPAL Nº 5.099, DE 27 DE OUT..."
4,2020-11-03,t Ã â 7 Z t Ã â 7 Z t Ã â 7 Z t Ã â 7 Z Prefei...
5,2020-10-19,"t Ã â 7 Z Fi ª PORTARIA Nº 11.963, DE 15 DE OU..."
6,2020-10-27,Prefeitura Municipal da Estância Turística de ...
7,2020-10-26,"t Ã â 7 Z DECRETO Nº 9.066, DE 20 DE OUTUBRO D..."
8,2020-10-22,t Ã â 7 Z VANA&%&Ã&(X«RX&. medidas de na Câmar...
9,2020-10-23,"t Ã â 7 Z LA DECRETO Nº 9.058, DE O5 DE OUTUBR..."


# Classificação dos Textos

## 1 - Quais as palavras mais frequentes por diário oficial

In [2]:
import spacy
from collections import Counter
from spacy.lang.pt import Portuguese
from spacy.lang.pt.stop_words import STOP_WORDS

In [3]:
#criando objeto para nlp em pt
nlp = Portuguese()
STOP_WORDS.add('e')
STOP_WORDS.add('a')
STOP_WORDS.add('o')

In [24]:
#teste
text = df_gazettes['content'][4].lower()
doc = nlp(text)

In [30]:
tipo = "licitação pregão concorrência inexigibilidade leilão carta-convite lei orçamento decreto projeto concurso edital"
area = "educação saúde finanças esporte lazer segurança ambiente social mobilidade"
nlp_tipo = nlp(tipo)
nlp_area = nlp(area)

In [31]:
doc_tokens = [token for token in doc if (token.text.lower() not in STOP_WORDS) and token.is_punct != True and token.like_num != True and token.is_alpha == True]
doc_texts = [token.text.lower() for token in doc if (token.text.lower() not in STOP_WORDS) and token.is_punct != True and token.like_num != True and token.is_alpha == True]
tipo_tokens = [token for token in nlp_tipo if token.is_punct != True]
area_tokens = [token for token in nlp_area if token.is_punct != True]

[educação, saúde, finanças, esporte, lazer, segurança, ambiente, social]

In [65]:
def gera_token(gazette):
    doc = nlp(gazette)
    doc_token =  [token for token in doc if (token.text.lower() not in STOP_WORDS) and token.is_punct != True and token.like_num != True and token.is_alpha == True]
    return doc_token
def classificacao(token_diario,token_tipo,token_area):
    tags_tipo = []
    tags_area = []
    for word in token_diario:
        for tipo in token_tipo:
            if word.similarity(tipo) >=0.9:
                #print(f"{word} - {tipo} = {word.similarity(tipo)}")
                tags_tipo.append(tipo)
            #else:
            #    tags_tipo.append("Outros")
        for area in token_area:
            if word.similarity(area) >=0.9:
                #print(f"{word} - {tipo} = {word.similarity(tipo)}")
                tags_area.append(area)
            #else:
             #   tags_area.append("Outros")
    return (tags_tipo,tags_area)

tipo = "licitação pregão concorrência inexigibilidade leilão carta-convite lei orçamento decreto projeto concurso edital"
area = "educação saúde finanças esporte lazer segurança ambiente social mobilidade"
#nlp_tipo = nlp(tipo)
#nlp_area = nlp(area)
tipo_tokens = gera_token(tipo)
area_tokens = gera_token(area)

In [66]:
tags = []
for idx,gazette in df_gazettes.iterrows():
    #df_gazettes['tipo'] = 
    tags.append(set(classificacao(gera_token(gazette['content']),tipo_tokens,area_tokens)[1]))
    #df_gazettes['area'] = classificacao(gera_token(gazette['content']),tipo_tokens,area_tokens)[1]

In [67]:
tags

[set(),
 {saúde},
 set(),
 {segurança},
 set(),
 set(),
 {segurança},
 {educação, saúde, esporte, lazer, segurança, ambiente, social, mobilidade},
 set(),
 {saúde},
 {educação, saúde, segurança, ambiente, social},
 {saúde, segurança},
 set(),
 set(),
 set(),
 set(),
 {saúde, lazer, segurança, mobilidade},
 set(),
 set(),
 {esporte, segurança, ambiente, social},
 set(),
 {segurança},
 set(),
 set(),
 set(),
 {segurança},
 set(),
 set(),
 set()]

In [577]:
# five most common tokens
word_freq = Counter(doc_texts)
common_words = word_freq.most_common(5)

In [526]:
tf = [val/total for val in word_freq.values()]
tf = dict(zip(word_freq.keys(),tf))

In [530]:
Counter(tf).most_common(5)

[('mun', 0.11703703703703704),
 ('secretaria', 0.1037037037037037),
 ('municipal', 0.08148148148148149),
 ('dependencias', 0.08),
 ('nº', 0.045925925925925926)]

In [545]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instanciar o objeto TFIDF 
vectorizer = TfidfVectorizer()
vectorizer.fit(doc_texts)
vectorizer.vocabulary_

{'decreto': 151,
 'nº': 417,
 'outubro': 436,
 'dispõe': 195,
 'abertura': 3,
 'crédito': 138,
 'adicional': 10,
 'suplementar': 588,
 'orçamento': 432,
 'vigente': 639,
 'marcubs': 374,
 'augustin': 56,
 'soliva': 581,
 'prefeito': 467,
 'municipio': 398,
 'estância': 238,
 'turística': 618,
 'guaratinguetá': 306,
 'uso': 628,
 'atribúlgões': 53,
 'cargo': 75,
 'especialmente': 226,
 'conformidade': 114,
 'lei': 355,
 'municipal': 396,
 'novembro': 412,
 'decret': 149,
 'fica': 264,
 'aberto': 2,
 'secretaria': 552,
 'fazenda': 258,
 'selã': 564,
 'setencentos': 574,
 'reais': 504,
 'centavos': 81,
 'abaixo': 1,
 'discriminadas': 194,
 'suplement': 587,
 'eguintes': 206,
 'dotações': 198,
 'fieha': 269,
 'unidade': 625,
 'orçamentária': 433,
 'executora': 250,
 'conta': 122,
 'mun': 390,
 'deplanejamento': 165,
 'coord': 130,
 'dependencias': 163,
 'deadministração': 142,
 'dejustiça': 158,
 'cidadanta': 88,
 'sec': 550,
 'desegurança': 170,
 'emobilidade': 210,
 'jurbana': 350,
 'dee

In [543]:
[print(x[0]) for x in Counter(tf).most_common(5)]

mun
secretaria
municipal
dependencias
nº


[None, None, None, None, None]